<a href="https://colab.research.google.com/github/wisesprout/dl-selfie-filter/blob/main/CodestatesProject1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GET DATA

In [4]:
from google.colab import files
import pandas as pd

In [1]:
# get data from kaggle
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list
!kaggle competitions download -c facial-keypoints-detection

Saving kaggle.json to kaggle.json
ref                                                         title                                             size  lastUpdated          downloadCount  
----------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  
prasertk/netflix-subscription-price-in-different-countries  Netflix subscription fee in different countries    3KB  2022-01-15 07:06:09           4094  
nkitgupta/jigsaw-regression-based-data                      Jigsaw Regression Based Data                       3GB  2022-02-05 20:51:56            955  
yasserh/wine-quality-dataset                                Wine Quality Dataset                              21KB  2022-01-15 19:15:11           4424  
majyhain/height-of-male-and-female-by-country-2022          Height of Male and Female by Country 2022          4KB  2022-02-02 00:40:19           1592  
sanjeetsinghnaik/top-1000-highest-grossing-movie

In [3]:
# unzip zipped files
! unzip test.zip
! unzip training.zip

Archive:  test.zip
  inflating: test.csv                
Archive:  training.zip
  inflating: training.csv            


In [5]:
train = pd.read_csv('/content/training.csv')
test = pd.read_csv('/content/test.csv')

# PREPROCESSING DATA

# BUILD MODEL

# FIT MODEL

# TEST MODEL

# LIVE CAM TEST